Lucas-kanade method (to track only few points)

In [7]:
import numpy as np
import cv2

# corner params for lucas kanade method
corner_track_param = dict(maxCorners = 20, qualityLevel=0.1, minDistance=3, blockSize=3)

# lucas parameters
lk_params = dict(winSize=(100, 100), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TermCriteria_COUNT, 5, 0.01))

In [8]:
cap = cv2.VideoCapture(0) # read from webcam
ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# points to track
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask=None, **corner_track_param)
mask = np.zeros_like(prev_frame)

while True:
    ret, curr_frame = cap.read()
    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
    
    # Optical flow methos: Lucas kanade
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prevPts, None, **lk_params)
    
    good_new = nextPts[status == 1]
    good_prev = prevPts[status == 1]
    
    for i, (new, prev) in enumerate(zip(good_new, good_prev)):
        x_new, y_new = new.ravel()
        x_prev, y_prev = prev.ravel()
        
        mask = cv2.line(mask, (int(x_new), int(y_new)), (int(x_prev), int(y_prev)), (0, 255, 0), 3)
        curr_gray = cv2.circle(curr_gray, (int(x_new), int(y_new)), 8, (0, 0, 255), -1)
        
    img = cv2.add(curr_frame, mask)
    cv2.imshow('tracking', img)
    
    k = cv2.waitKey(30) & 0xFF  # press esc to close
    if k == 27:
        break
    
    prev_gray = curr_gray.copy()
    prevPts = good_new.reshape(-1, 1, 2)
    
cap.release()
cv2.destroyAllWindows()

    
    